In [1]:
import pandas as pd
df = pd.read_csv("/content/Corona_NLP.csv")
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
df = df.drop(columns=['UserName', 'ScreenName', 'Location', 'TweetAt',])
df

In [3]:
sentiment = pd.get_dummies(df.Sentiment)
df_tweet = pd.concat([df, sentiment], axis=1)
df_tweet = df_tweet.drop(columns='Sentiment')
df_tweet

,OriginalTweet,Extremely Negative,Extremely Positive,Negative,Neutral,Positive
0,TRENDING: New Yorkers encounter empty supermar...,1,0,0,0,0
1,When I couldn't find hand sanitizer at Fred Me...,0,0,0,0,1
2,Find out how you can protect yourself and love...,0,1,0,0,0
3,#Panic buying hits #NewYork City as anxious sh...,0,0,1,0,0
4,#toiletpaper #dunnypaper #coronavirus #coronav...,0,0,0,1,0
...,...,...,...,...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,0,0,0,0,1
3794,Did you panic buy a lot of non-perishable item...,0,0,1,0,0
3795,Asst Prof of Economics @cconces was on @NBCPhi...,0,0,0,1,0
3796,Gov need to do somethings instead of biar je r...,1,0,0,0,0


In [4]:
tweets = df_tweet['OriginalTweet'].values
label = df_tweet[['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral', 'Positive']].values

In [5]:
from sklearn.model_selection import train_test_split
tweet_train, tweet_test, label_train, label_test = train_test_split(tweets, label, test_size=0.2)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(tweet_train)
# tokenizer.fit_on_texts(tweet_test)

sequence_train = tokenizer.texts_to_sequences(tweet_train)
sequence_test = tokenizer.texts_to_sequences(tweet_test)

padded_train = pad_sequences(sequence_train)
padded_test = pad_sequences(sequence_test)

In [7]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
num_epochs = 30
history = model.fit(padded_train, label_train, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
95/95 - 7s - loss: 1.5799 - accuracy: 0.2670 - val_loss: 1.5857 - val_accuracy: 0.2526 - 7s/epoch - 75ms/step
Epoch 2/30
95/95 - 4s - loss: 1.5153 - accuracy: 0.3005 - val_loss: 1.4915 - val_accuracy: 0.3276 - 4s/epoch - 37ms/step
Epoch 3/30
95/95 - 3s - loss: 1.3130 - accuracy: 0.3983 - val_loss: 1.5124 - val_accuracy: 0.3618 - 3s/epoch - 36ms/step
Epoch 4/30
95/95 - 3s - loss: 1.0871 - accuracy: 0.4987 - val_loss: 1.6173 - val_accuracy: 0.3434 - 3s/epoch - 36ms/step
Epoch 5/30
95/95 - 4s - loss: 0.8522 - accuracy: 0.6267 - val_loss: 1.9534 - val_accuracy: 0.3658 - 4s/epoch - 37ms/step
Epoch 6/30
95/95 - 5s - loss: 0.6708 - accuracy: 0.7248 - val_loss: 2.1761 - val_accuracy: 0.3553 - 5s/epoch - 54ms/step
Epoch 7/30
95/95 - 6s - loss: 0.5389 - accuracy: 0.7795 - val_loss: 2.4752 - val_accuracy: 0.3697 - 6s/epoch - 64ms/step
Epoch 8/30
95/95 - 4s - loss: 0.4462 - accuracy: 0.8249 - val_loss: 3.0646 - val_accuracy: 0.3553 - 4s/epoch - 39ms/step
Epoch 9/30
95/95 - 4s - loss: 0.